# Tutorial 2: RF Pulses

In this tutorial, we explore the different types of radio-frequency (RF) pulses available in
PyPulseq. RF pulses are fundamental building blocks of every MRI pulse sequence — they are
used to excite, refocus, invert, or saturate the magnetization.

## What you will learn

1. How to create different RF pulse shapes (block, sinc, Gaussian)
2. How pulse parameters affect the waveform (duration, time-bandwidth product, apodization)
3. How to make RF pulses slice-selective using gradients
4. How to control flip angle, phase offset, and frequency offset
5. How to specify the purpose of an RF pulse (`use` parameter)
6. How to assemble RF pulses with gradients into a sequence

In [ ]:
import importlib

if not importlib.util.find_spec('pypulseq'):
    %pip install pypulseq

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import pypulseq as pp

In [ ]:
system = pp.Opts(
    max_grad=28,
    grad_unit='mT/m',
    max_slew=150,
    slew_unit='T/m/s',
    rf_ringdown_time=20e-6,
    rf_dead_time=100e-6,
    adc_dead_time=10e-6,
)

Before we start, let's define a small helper function to plot the RF pulse waveform. This
will allow us to quickly visualize and compare different RF pulses throughout this tutorial.

In [ ]:
def plot_rf(rf, title='', ax=None):
    """Plot the magnitude and phase of an RF pulse waveform.

    The plot includes the full event timing: dead time before the pulse,
    the pulse waveform itself, and the ringdown time after the pulse.
    """
    if ax is None:
        _, ax = plt.subplots(2, 1, figsize=(8, 4), sharex=True)

    # Build the full time axis including dead time and ringdown
    t_full = np.concatenate(
        ([0], [rf.delay], rf.delay + rf.t, [rf.delay + rf.shape_dur], [rf.delay + rf.shape_dur + rf.ringdown_time])
    )
    signal_full = np.concatenate(([0], [0], rf.signal, [0], [0]))

    t_ms = t_full * 1e3  # Convert to ms

    ax[0].plot(t_ms, np.abs(signal_full), 'b-', linewidth=1.5)
    ax[0].set_ylabel('Amplitude (Hz)')
    ax[0].set_title(title)
    ax[0].grid(True, alpha=0.3)

    ax[1].plot(t_ms, np.angle(signal_full, deg=True), 'r-', linewidth=1.5)
    ax[1].set_ylabel('Phase (°)')
    ax[1].set_xlabel('Time (ms)')
    ax[1].grid(True, alpha=0.3)

    return ax

---
## 1. Block (Rectangular) Pulses

The simplest RF pulse is a **block pulse** (also called a rectangular or hard pulse). It has
a constant amplitude for its entire duration. Block pulses are non-selective, meaning they
excite all spins equally regardless of their spatial position.

We already used `pp.make_block_pulse` in Tutorial 1. Let's now look at it in more detail.

In [ ]:
rf_block = pp.make_block_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=1e-3,
    system=system,
    use='excitation',
)

print(f'RF amplitude: {np.max(np.abs(rf_block.signal)):.1f} Hz')
print(f'Shape duration: {rf_block.shape_dur * 1e3:.2f} ms')
print(f'Total duration (incl. dead time + ringdown): {pp.calc_duration(rf_block) * 1e3:.2f} ms')

In [ ]:
%matplotlib inline
plot_rf(rf_block, title='90° Block Pulse (1 ms)')
plt.tight_layout()
plt.show()

As expected, the block pulse has a constant amplitude and zero phase. The amplitude is
determined by the flip angle and the duration: a shorter pulse requires a higher amplitude
to achieve the same flip angle.

### 1.1 Effect of flip angle on amplitude

Let's compare block pulses with different flip angles but the same duration.

In [ ]:
flip_angles_deg = [10, 30, 60, 90, 180]

fig, ax = plt.subplots(1, 1, figsize=(8, 3))
for fa_deg in flip_angles_deg:
    rf = pp.make_block_pulse(
        flip_angle=np.deg2rad(fa_deg),
        delay=system.rf_dead_time,
        duration=1e-3,
        system=system,
    )
    rf_signal = np.concatenate(([0], rf.signal, [0]))
    rf_time = np.concatenate(([0], rf.t, [rf.t[-1]]))
    ax.plot(rf_time * 1e3, rf_signal, label=f'{fa_deg}°', linewidth=1.5)

ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (Hz)')
ax.set_title('Block Pulses: Effect of Flip Angle')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

The amplitude scales linearly with the flip angle. A 180° pulse has exactly twice the
amplitude of a 90° pulse of the same duration.

---
## 2. Sinc Pulses

In practice, block pulses are rarely used for slice-selective excitation because their
frequency profile (the Fourier transform of the pulse shape) is a sinc function, which has
poor slice selectivity.

Instead, we use **sinc-shaped** RF pulses, whose Fourier transform approximates a
rectangular frequency profile — exactly what we need for sharp slice selection.

The key parameters of a sinc pulse are:

- **`duration`** — the total pulse duration
- **`time_bw_product`** (TBW) — the time-bandwidth product, which controls the number of
  zero-crossings and thus the sharpness of the slice profile. Higher TBW = sharper profile
  but more sidelobes.
- **`apodization`** — a windowing factor (0 to 1) that suppresses sidelobes at the cost of
  a slightly wider transition band. 0 = no apodization (pure sinc), 1 = full Hanning window.

In [ ]:
rf_sinc = pp.make_sinc_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=3e-3,
    time_bw_product=4,
    apodization=0.5,
    system=system,
    use='excitation',
)

print(f'Peak amplitude: {np.max(np.abs(rf_sinc.signal)):.1f} Hz')
print(f'Shape duration: {rf_sinc.shape_dur * 1e3:.2f} ms')
print(f'Total duration: {pp.calc_duration(rf_sinc) * 1e3:.2f} ms')

In [ ]:
plot_rf(rf_sinc, title='90° Sinc Pulse (3 ms, TBW=4, apod=0.5)')
plt.tight_layout()
plt.show()

### 2.1 Effect of time-bandwidth product

The time-bandwidth product (TBW) determines the number of lobes in the sinc pulse. A higher
TBW results in more zero-crossings, which leads to a sharper slice profile but also requires
a longer pulse duration (or higher gradient amplitude) to maintain the same slice thickness.

In [ ]:
tbw_values = [2, 4, 8]

fig, axes = plt.subplots(1, len(tbw_values), figsize=(14, 3), sharey=True)
for ax, tbw in zip(axes, tbw_values):
    rf = pp.make_sinc_pulse(
        flip_angle=np.deg2rad(90),
        delay=system.rf_dead_time,
        duration=3e-3,
        time_bw_product=tbw,
        apodization=0.0,
        system=system,
    )
    ax.plot(rf.t * 1e3, np.abs(rf.signal), 'b-', linewidth=1.5)
    ax.set_title(f'TBW = {tbw}')
    ax.set_xlabel('Time (ms)')
    ax.grid(True, alpha=0.3)

axes[0].set_ylabel('Amplitude (Hz)')
fig.suptitle('Sinc Pulses: Effect of Time-Bandwidth Product (no apodization)', y=1.02)
plt.tight_layout()
plt.show()

### 2.2 Effect of apodization

Apodization applies a Hanning window to the sinc pulse. This suppresses the sidelobes,
reducing ringing artifacts in the slice profile at the cost of a slightly wider transition
band. The `apodization` parameter ranges from 0 (no windowing) to 1 (full Hanning window).
A value of 0.5 is commonly used as a good compromise.

In [ ]:
apodization_values = [0.0, 0.25, 0.5, 0.75]

fig, ax = plt.subplots(1, 1, figsize=(8, 3))
for apod in apodization_values:
    rf = pp.make_sinc_pulse(
        flip_angle=np.deg2rad(90),
        delay=system.rf_dead_time,
        duration=3e-3,
        time_bw_product=4,
        apodization=apod,
        system=system,
    )
    ax.plot(rf.t * 1e3, np.abs(rf.signal), label=f'apodization = {apod}', linewidth=1.5)

ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (Hz)')
ax.set_title('Sinc Pulses: Effect of Apodization (TBW=4)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Notice how increasing the apodization smoothly tapers the sidelobes towards zero, resulting
in a smoother pulse envelope.

---
## 3. Gaussian Pulses

**Gaussian pulses** have a bell-shaped envelope. They are smooth and have no sidelobes,
which makes them useful in situations where a smooth frequency response is more important
than a sharp slice profile (e.g., fat saturation, magnetization preparation).

The `pp.make_gauss_pulse` function has a very similar interface to `pp.make_sinc_pulse`.

In [ ]:
rf_gauss = pp.make_gauss_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=3e-3,
    time_bw_product=4,
    system=system,
    use='excitation',
)

print(f'Peak amplitude: {np.max(np.abs(rf_gauss.signal)):.1f} Hz')
print(f'Shape duration: {rf_gauss.shape_dur * 1e3:.2f} ms')

In [ ]:
plot_rf(rf_gauss, title='90° Gaussian Pulse (3 ms, TBW=4)')
plt.tight_layout()
plt.show()

### 3.1 Comparing pulse shapes

Let's compare all three pulse shapes side by side with the same duration and flip angle.

In [ ]:
duration = 3e-3
flip_angle = np.deg2rad(90)

rf_block_cmp = pp.make_block_pulse(flip_angle=flip_angle, delay=system.rf_dead_time, duration=duration, system=system)
rf_sinc_cmp = pp.make_sinc_pulse(
    flip_angle=flip_angle,
    delay=system.rf_dead_time,
    duration=duration,
    time_bw_product=4,
    apodization=0.5,
    system=system,
)
rf_gauss_cmp = pp.make_gauss_pulse(
    flip_angle=flip_angle, delay=system.rf_dead_time, duration=duration, time_bw_product=4, system=system
)

fig, ax = plt.subplots(1, 1, figsize=(8, 3))
ax.plot(
    np.concatenate(([0], rf_block_cmp.t, [rf_block_cmp.t[-1]])) * 1e3,
    np.concatenate(([0], np.abs(rf_block_cmp.signal), [0])),
    label='Block',
    linewidth=1.5,
)
ax.plot(rf_sinc_cmp.t * 1e3, np.abs(rf_sinc_cmp.signal), label='Sinc (TBW=4, apod=0.5)', linewidth=1.5)
ax.plot(rf_gauss_cmp.t * 1e3, np.abs(rf_gauss_cmp.signal), label='Gaussian (TBW=4)', linewidth=1.5)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (Hz)')
ax.set_title('Comparison of RF Pulse Shapes (90°, 3 ms)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Each shape has different trade-offs:

| Shape | Slice profile | Sidelobes | Typical use |
|---|---|---|---|
| Block | sinc-shaped (poor) | Many | Non-selective excitation |
| Sinc | Approximately rectangular | Controllable via TBW and apodization | Slice-selective excitation/refocusing |
| Gaussian | Gaussian (smooth) | None | Fat saturation, preparation |

---
## 4. Slice-Selective RF Pulses

To excite only a specific slice of the object, we play a **slice-selection gradient** during
the RF pulse. The gradient creates a linear mapping between spatial position and resonance
frequency. Combined with the frequency-selective RF pulse, only spins within the desired
slice are affected.

After the RF pulse, the spins within the slice have accumulated different phases due to the
slice-selection gradient. A **rephasing gradient** (also called rewinder) is applied to
refocus this phase dispersion.

In PyPulseq, you can request the slice-selection gradient and its rephaser by setting
`return_gz=True` and providing a `slice_thickness`.

In [ ]:
rf_ss, gz_ss, gz_reph = pp.make_sinc_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=3e-3,
    time_bw_product=4,
    apodization=0.5,
    slice_thickness=5e-3,
    return_gz=True,
    system=system,
    use='excitation',
)

print(f'RF shape duration: {rf_ss.shape_dur * 1e3:.2f} ms')
print(f'RF delay (adjusted due to rf_dead_time): {rf_ss.delay * 1e6:.0f} µs')
print(f'RF total duration (including dead/ringdown time): {pp.calc_duration(rf_ss) * 1e3:.2f} ms')
print(f'Slice-select gradient delay: {gz_ss.delay * 1e6:.0f} µs')
print(f'Slice-select gradient rise time: {gz_ss.rise_time * 1e6:.0f} µs')
print(f'Slice-select gradient flat time: {gz_ss.flat_time * 1e3:.2f} ms')
print(f'Slice-select gradient total duration: {pp.calc_duration(gz_ss) * 1e3:.2f} ms')
print(f'Rephasing gradient duration: {pp.calc_duration(gz_reph) * 1e3:.2f} ms')

### 4.1 Visualizing the slice-selective excitation block

Let's build a minimal sequence with just the slice-selective RF pulse and its gradients to
visualize how they are arranged in time.

In [ ]:
seq = pp.Sequence(system)
seq.add_block(rf_ss, gz_ss)
seq.add_block(gz_reph)

seq.plot(grad_disp='mT/m')
plt.show()

In the plot above, you can see:

- **Block 1**: The RF pulse (top) plays simultaneously with the slice-selection gradient on
  the z-axis (bottom). The gradient has a trapezoidal shape with a flat top during the RF
  pulse and ramps on either side.
- **Block 2**: The rephasing gradient on the z-axis, which has the opposite polarity to
  compensate for the phase accumulated during the second half of the slice-selection gradient.

Note how the RF pulse's and gradient's `delay` were automatically adjusted to account for both, the RF dead time and the gradient's
rise time, ensuring that the RF pulse starts when the gradient has reached its flat top.

### 4.2 Effect of slice thickness and gradient units

The slice thickness is controlled by the amplitude of the slice-selection gradient: a
thinner slice requires a stronger gradient. Let's compare different slice thicknesses.

**A note on gradient units in Pulseq:** Internally, (Py)Pulseq stores all gradient
amplitudes in **Hz/m** rather than the more familiar **mT/m**. This is actually very
convenient because the Larmor equation directly links frequency to position:

$$f = \gamma \cdot G \cdot x$$

where $G$ is the gradient amplitude in T/m and $\gamma$ is the gyromagnetic ratio (Hz/T).
By storing gradients in Hz/m, we can directly compute frequencies without extra conversion
factors. The relationship between the two units is:

$$G_\text{[Hz/m]} = \gamma \cdot G_\text{[T/m]} = \gamma \cdot G_\text{[mT/m]} \cdot 10^{-3}$$

To convert back to mT/m (e.g., for display), we divide by `system.gamma` and multiply by
1000:

$$G_\text{[mT/m]} = \frac{G_\text{[Hz/m]}}{\gamma} \cdot 10^3$$

In [ ]:
slice_thicknesses_mm = [2, 5, 10]

for st_mm in slice_thicknesses_mm:
    _, gz, gzr = pp.make_sinc_pulse(
        flip_angle=np.deg2rad(90),
        delay=system.rf_dead_time,
        duration=3e-3,
        time_bw_product=4,
        apodization=0.5,
        slice_thickness=st_mm * 1e-3,
        return_gz=True,
        system=system,
    )
    grad_mTm = gz.amplitude / system.gamma * 1e3
    print(
        f'Slice thickness: {st_mm:2d} mm -> '
        f'Gradient: {gz.amplitude:10.1f} Hz/m = {grad_mTm:6.2f} mT/m, '
        f'Rephaser duration: {pp.calc_duration(gzr) * 1e3:.2f} ms'
    )

print(
    f'\nMax gradient amplitude of our system: {system.max_grad:.1f} Hz/m = {system.max_grad / system.gamma * 1e3:.1f} mT/m'
)

As expected, thinner slices require stronger gradients. The rephasing gradient duration also
changes because its area must match the area of the second half of the slice-selection
gradient plus the ramp-down area.


### 4.3 What happens when the gradient limit is exceeded?

If we request a very thin slice, the required gradient amplitude may exceed the hardware
limit (`max_grad`). In that case, PyPulseq raises an error. Let's try a 0.5 mm slice with
our current system limits (28 mT/m):

In [ ]:
try:
    _, gz_thin, _ = pp.make_sinc_pulse(
        flip_angle=np.deg2rad(90),
        delay=system.rf_dead_time,
        duration=3e-3,
        time_bw_product=4,
        apodization=0.5,
        slice_thickness=0.5e-3,  # 0.5 mm — very thin!
        return_gz=True,
        system=system,
    )
except ValueError as e:
    # Calculate what the required gradient would be
    bandwidth = 4 / 3e-3  # TBW / duration
    required_grad = bandwidth / 0.5e-3  # Hz/m
    required_grad_mTm = required_grad / system.gamma * 1e3
    print(f'Error: {e}')
    print(f'\nRequired gradient amplitude:   {required_grad:.1f} Hz/m = {required_grad_mTm:.1f} mT/m')
    print(
        f'System max gradient amplitude: {system.max_grad:.1f} Hz/m = {system.max_grad / system.gamma * 1e3:.1f} mT/m'
    )

To achieve thinner slices without exceeding the gradient limit, you can either increase the
pulse duration (which reduces the required bandwidth) or increase the `max_grad` in your
system limits (if the scanner hardware supports it).

### 4.4 Gaussian slice-selective pulse

Slice selection also works with Gaussian pulses. The interface is identical.

In [ ]:
rf_gauss_ss, gz_gauss_ss, gz_gauss_reph = pp.make_gauss_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=3e-3,
    time_bw_product=4,
    slice_thickness=5e-3,
    return_gz=True,
    system=system,
    use='excitation',
)

seq = pp.Sequence(system)
seq.add_block(rf_gauss_ss, gz_gauss_ss)
seq.add_block(gz_gauss_reph)

seq.plot()
plt.show()

---
## 5. Flip Angle, Phase Offset, and Frequency Offset

Every RF pulse in PyPulseq can be configured with three important parameters that control
how it interacts with the magnetization:

- **`flip_angle`** — the angle (in radians) by which the magnetization is rotated
- **`phase_offset`** — the phase of the RF pulse (in radians), which determines the axis of
  rotation in the transverse plane
- **`freq_offset`** — a frequency offset (in Hz) applied to the RF pulse, which can be used
  to shift the excitation to a different frequency (e.g., for multi-slice imaging or
  fat/water-selective excitation)

### 5.1 Phase offset

The phase offset rotates the axis of excitation in the transverse plane. For example, a
phase offset of 0 excites along the x-axis, while π/2 excites along the y-axis. This is
commonly used in RF spoiling schemes and for refocusing pulses.

Let's create sinc pulses with different phase offsets and visualize the complex waveform.

In [ ]:
phase_offsets = [0, np.pi / 4, np.pi / 2, np.pi]
phase_labels = ['0', 'π/4', 'π/2', 'π']

fig, axes = plt.subplots(2, len(phase_offsets), figsize=(16, 4), sharex=True, sharey=True)

for i, (phase, label) in enumerate(zip(phase_offsets, phase_labels)):
    rf = pp.make_sinc_pulse(
        flip_angle=np.deg2rad(90),
        delay=system.rf_dead_time,
        duration=3e-3,
        time_bw_product=4,
        apodization=0.5,
        phase_offset=phase,
        system=system,
    )
    t_ms = rf.t * 1e3
    # Apply the phase offset to the signal for visualization.
    # PyPulseq stores phase_offset separately (applied at playback), so the
    # signal array itself is always real-valued. We rotate it here to show
    # the effect of the phase offset on the complex waveform.
    signal_rotated = rf.signal * np.exp(1j * rf.phase_offset)
    axes[0, i].plot(t_ms, np.real(signal_rotated), 'b-', linewidth=1.5)
    axes[0, i].set_title(f'Phase = {label}')
    axes[0, i].grid(True, alpha=0.3)
    axes[1, i].plot(t_ms, np.imag(signal_rotated), 'r-', linewidth=1.5)
    axes[1, i].set_xlabel('Time (ms)')
    axes[1, i].grid(True, alpha=0.3)

axes[0, 0].set_ylabel('Real part (Hz)')
axes[1, 0].set_ylabel('Imaginary part (Hz)')
fig.suptitle('Sinc Pulses with Different Phase Offsets', y=1.02)
plt.tight_layout()
plt.show()

The phase offset rotates the complex RF waveform. At phase = 0, the signal is purely real.
At phase = π/2, it becomes purely imaginary. At intermediate values, both real and imaginary
components are present.

### 5.2 Frequency offset

The frequency offset shifts the center frequency of the RF pulse. This is useful for:

- **Multi-slice imaging**: exciting different slices by offsetting the RF frequency while
  keeping the same slice-selection gradient
- **Fat/water-selective excitation**: targeting specific spectral components

The frequency offset is stored as a property of the RF event and is applied during playback
on the scanner. It does **not** change the waveform shape itself.

Let's demonstrate multi-slice excitation by creating RF pulses with different frequency
offsets.

In [ ]:
slice_positions_mm = [-10, -5, 0, 5, 10]  # 5 slices, 5 mm apart

# Create the slice-selective pulse (we need the gradient amplitude to calculate freq offsets)
rf_base, gz_base, gz_reph_base = pp.make_sinc_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=3e-3,
    time_bw_product=4,
    apodization=0.5,
    slice_thickness=5e-3,
    return_gz=True,
    system=system,
    use='excitation',
)

# The frequency offset for a given slice position is:
# freq_offset = gradient_amplitude * slice_position
for pos_mm in slice_positions_mm:
    freq_offset = gz_base.amplitude * pos_mm * 1e-3  # Convert mm to m
    print(f'Slice at {pos_mm:+3d} mm -> freq_offset = {freq_offset:+8.1f} Hz')

---
## 6. The `use` Parameter

Every RF pulse in (Py)Pulseq has a `use` parameter that describes its purpose in the sequence.
Since Pulseq file format version 1.5, this metadata is stored in the `.seq` file and is thus still available 
when reading the sequence. 

The supported values are:

| Value | Description |
|---|---|
| `'excitation'` | Excitation pulse (tips magnetization into the transverse plane) |
| `'refocusing'` | Refocusing pulse (e.g., 180° pulse in spin echo sequences) |
| `'inversion'` | Inversion pulse (e.g., 180° pulse for inversion recovery) |
| `'saturation'` | Saturation pulse (e.g., fat saturation, CEST) |
| `'preparation'` | Preparation pulse (e.g., T2 preparation) |
| `'other'` | Other purpose |
| `'undefined'` | Default if not specified |

Let's create examples of the most common use cases.

### 6.1 Excitation pulse

An excitation pulse tips the longitudinal magnetization into the transverse plane. It
typically has a flip angle between 1° and 90°.

In [ ]:
rf_excitation, gz_excitation, gz_excitation_reph = pp.make_sinc_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=3e-3,
    time_bw_product=4,
    apodization=0.5,
    slice_thickness=5e-3,
    return_gz=True,
    system=system,
    use='excitation',
)

print(f'Excitation pulse: flip angle = 90°, use = {rf_excitation.use}')

### 6.2 Refocusing pulse

A refocusing pulse is a 180° pulse used in spin echo sequences to refocus the transverse
magnetization. It is typically played with a phase offset of π/2 relative to the excitation
pulse (i.e., along the y-axis if the excitation is along the x-axis).

In [ ]:
rf_refocusing, gz_refocusing, gz_refocusing_reph = pp.make_sinc_pulse(
    flip_angle=np.deg2rad(180),
    delay=system.rf_dead_time,
    duration=3e-3,
    time_bw_product=4,
    apodization=0.5,
    phase_offset=np.pi / 2,
    slice_thickness=5e-3,
    return_gz=True,
    system=system,
    use='refocusing',
)

print(f'Refocusing pulse: flip angle = 180°, phase = π/2, use = {rf_refocusing.use}')

### 6.3 Inversion pulse

An inversion pulse flips the longitudinal magnetization by 180°. Unlike a refocusing pulse,
it acts on the longitudinal component and is used in inversion recovery sequences (e.g.,
MPRAGE, STIR, FLAIR).

For inversion, adiabatic pulses are often preferred because they achieve a uniform 180° flip
even in the presence of B1 inhomogeneities. PyPulseq provides `pp.make_adiabatic_pulse` for
this purpose, supporting hyperbolic secant (`hypsec`) and WURST pulse types.

In [ ]:
rf_inversion = pp.make_adiabatic_pulse(
    pulse_type='hypsec',
    delay=system.rf_dead_time,
    duration=10e-3,
    system=system,
    use='inversion',
)

print(f'Inversion pulse type: hypsec, use = {rf_inversion.use}')
print(f'Duration: {rf_inversion.shape_dur * 1e3:.1f} ms')

In [ ]:
plot_rf(rf_inversion, title='Adiabatic Inversion Pulse (Hyperbolic Secant, 10 ms)')
plt.tight_layout()
plt.show()

Unlike the previous pulses, the adiabatic pulse has a **non-zero phase** that varies across
the pulse duration. This frequency sweep is what makes adiabatic pulses robust to B1
inhomogeneities. For more details about adiabatic pulses in general, see for example [https://mriquestions.com/adiabatic-excitation.html](https://mriquestions.com/adiabatic-excitation.html).

### 6.4 Saturation pulse

A saturation pulse destroys the magnetization of a specific spectral component (e.g., fat).
It typically has a flip angle around 90° followed by a spoiler gradient. For frequency-selective
saturation (e.g., fat sat), a Gaussian pulse with a specific frequency offset is commonly
used. For (Py)Pulseq versions >= 1.5.0, the frequency offset can be specified in ppm (`freq_ppm`)
to make the RF pulse field strength independent.

In [ ]:
# Fat saturation pulse: 90° Gaussian pulse at the fat frequency offset (~-440 Hz at 3T)
rf_fatsat = pp.make_gauss_pulse(
    flip_angle=np.deg2rad(90),
    delay=system.rf_dead_time,
    duration=8e-3,
    time_bw_product=2,
    freq_ppm=-3.45,
    system=system,
    use='saturation',
)

print(f'Fat saturation pulse: use = {rf_fatsat.use}, freq_offset = {rf_fatsat.freq_ppm} ppm')

---
## Summary

In this tutorial, you learned about the different types of RF pulses available in PyPulseq:

| Concept | Function |
|---|---|
| Block (rectangular) pulse | `pp.make_block_pulse(flip_angle, duration, ...)` |
| Sinc pulse | `pp.make_sinc_pulse(flip_angle, duration, time_bw_product, apodization, ...)` |
| Gaussian pulse | `pp.make_gauss_pulse(flip_angle, duration, time_bw_product, ...)` |
| Adiabatic pulse | `pp.make_adiabatic_pulse(pulse_type, duration, ...)` |
| Slice selection | Set `return_gz=True` and `slice_thickness=...` |
| Phase offset | Set `phase_offset=...` (in radians) |
| Frequency offset | Set `freq_offset=...` (in Hz) |
| Pulse purpose | Set `use='excitation'`, `'refocusing'`, `'inversion'`, `'saturation'`, ... |

### Key takeaways

- **Block pulses** are simple but non-selective. Use them for hard-pulse excitation.
- **Sinc pulses** provide good slice selectivity. Tune `time_bw_product` and `apodization`
  for the desired profile sharpness.
- **Gaussian pulses** are smooth and sidelobe-free. Use them for spectral saturation or
  preparation.
- **Adiabatic pulses** are robust to B1 inhomogeneities. Use them for inversion.
- Set `return_gz=True` with a `slice_thickness` to automatically generate the slice-selection
  gradient and its rephaser.
- Use `phase_offset` and `freq_offset` to control the excitation axis and frequency.

## Next steps

In the next tutorial, we'll explore **gradient events** in detail — including trapezoids,
arbitrary gradients, and how to use them for spatial encoding.